# Part 3: Model Building

This notebook explores building the **customer churn model** and an **interpretability model** to explain each prediction.
In addition to making a prediction of whether a customer will churn, we will also be able to answer the question, "why are they expected to churn?"

The following work will look fairly standard to anyone having trained machine learning models using python Jupyter notebooks.
The CML platform provides a **fully capable Jupyter notebook environment** that data scientists know and love.

If you haven't yet, run through the initialization steps in the README file and Part 1. 
In Part 1, the data is imported into the `default.telco_churn` table in Hive. All data accesses fetch from Hive.


## Load data

We again start by creating a `SparkSession` to fetch the data using Spark SQL, only this time we convert to a pandas `DataFrame` since we saw earlier that there are only 7k records in the dataset.


In [ ]:
from pyspark.sql import SparkSession

spark = SparkSession\
    .builder\
    .appName("PythonSQL")\
    .master("local[*]")\
    .getOrCreate()

spark_df = spark.sql("SELECT * FROM default.telco_churn")
spark_df.printSchema()
df = spark_df.toPandas()

**Note:** If you don't have the Hive table, you can read the csv from the CML filesystem using pandas directly:

```python
import pandas as pd
import os

data_dir = '/home/cdsw'
df = pd.read_csv(os.path.join(data_dir, 'raw', 'WA_Fn-UseC_-Telco-Customer-Churn-.csv'))
```


## Basic feature engineering


Next we munge the data into appropriate types for later steps. 
In particular, we want to convert all the binary and string columns into pandas `Categorical` types.



In [ ]:
import subprocess, glob, sys
import dill  # a better pickle
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

data_dir = '/home/cdsw'

In [ ]:
idcol = 'customerID'  # ID column
labelcol = 'Churn'  # label column
cols = (('gender', True),  # (feature column, Categorical?)
        ('SeniorCitizen', True),
        ('Partner', True),
        ('Dependents', True),
        ('tenure', False),
        ('PhoneService', True),
        ('MultipleLines', True),
        ('InternetService', True),
        ('OnlineSecurity', True),
        ('OnlineBackup', True),
        ('DeviceProtection', True),
        ('TechSupport', True),
        ('StreamingTV', True),
        ('StreamingMovies', True),
        ('Contract', True),
        ('PaperlessBilling', True),
        ('PaymentMethod', True),
        ('MonthlyCharges', False),
        ('TotalCharges', False))

In [ ]:
df = df.replace(r'^\s$', np.nan, regex=True).dropna().reset_index()  # drop blank rows
df.index.name = 'id'  # name the index
data, labels = df.drop(labelcol, axis=1), df[labelcol]  # separate out the labels
data = data[[c for c, _ in cols]]  # only use the columns named in `cols`
data = data.replace({'SeniorCitizen': {1: 'Yes', 0: 'No'}})  # Change 1/0 to Yes/No to match the other binary features

# convert the categorical columns to pd.Categorical form
for col, iscat in cols:
    if iscat:
        data[col] = pd.Categorical(data[col])
labels = (labels == 'Yes')  # convert labels from str to bool

data.head()

## Machine learning model

This step follows a fairly standard ML workflow, which is to create a pipeline to:

* Encode the categorical features as numeric
* Normalize the numeric features
* Train a classification model using these processed features

We use *one-hot encoding*, *standardization*, and *logistic regression with cross-validation* for the three steps.
Then we can evaluate the model's performance.

Note: `CategoricalEncoder` and, later, `ExplainedModel` are helper classes pulled and edited from the original CFFL [interpretability report code](https://ff06-2020.fastforwardlabs.com/).
You can inspect `churnexplainer.py` to see what they do under the hood.
CML lets you continue to write modular code to keep things segregated and clean.

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegressionCV, LogisticRegression
from sklearn.compose import ColumnTransformer

from churnexplainer import CategoricalEncoder  # convert Categorical columns into numeric

In [ ]:
ce = CategoricalEncoder()
X = ce.fit_transform(data)  # Categorical columns now have values 0 to num_categories-1
y = labels.values
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=42)

cat_cols = list(ce.cat_columns_ix_.values())  # indices of the categorical columns (now numeric)
ct = ColumnTransformer(
    [('ohe', OneHotEncoder(), cat_cols)],
    remainder='passthrough'
)
clf = LogisticRegressionCV(cv=5,solver='lbfgs', max_iter=100)

In [ ]:
pipe = Pipeline([('ct', ct),  # 1. Encode the categorical features as numeric
                 ('scaler', StandardScaler()),  # 2. Normalize the numeric features
                 ('clf', clf)])  # 3. Train a classification model using these processed features
pipe.fit(X_train, y_train)
train_score = pipe.score(X_train, y_train)
test_score = pipe.score(X_test, y_test)
print("train",train_score)
print("test", test_score)    
print(classification_report(y_test, pipe.predict(X_test)))

### Compare with Random Forest
Just for a comparison, lets compare this model to a Random Forest model.
This is simpler since Random Forests do not need the categorical features encoded with a `OneHotEncoder`.

In [ ]:
from sklearn.ensemble import RandomForestClassifier
clf_rf = RandomForestClassifier(n_estimators=100)
pipe_rf = Pipeline([('scaler', StandardScaler()),
                 ('clf', clf_rf)])
pipe_rf.fit(X_train, y_train)
train_score = pipe_rf.score(X_train, y_train)
test_score = pipe_rf.score(X_test, y_test)
print("train",train_score)
print("test", test_score)
print(classification_report(y_test, pipe_rf.predict(X_test)))

### Plot ROC Curve

We can also generate an ROC Curve to visualize the model's performance and calculate the AUROC

In [ ]:
from sklearn.metrics import roc_curve
from sklearn.metrics import roc_auc_score
from matplotlib import pyplot

logistic_regression_probabilities = pipe.predict_proba(X_test)
logistic_regression_probabilities = logistic_regression_probabilities[:, 1]
logistic_regression_auc = roc_auc_score(y_test, logistic_regression_probabilities)
print('Logistic: AUROC=%.3f' % (logistic_regression_auc))
logistic_regression_fpr, logistic_regression_tpr, _ = roc_curve(y_test, logistic_regression_probabilities)
pyplot.plot(logistic_regression_fpr, logistic_regression_tpr, label='Logistic')
pyplot.show()

We find an AUC of 0.83. Not bad for a quick exercise without fine tuning.


## Interpretability model
We use [lime](https://github.com/marcotcr/lime) (Local Interpretable Model-Agnostic Explanations) to explain the predictions.
It is a method of determining which feature has the greatest effect on the predicted value,
and is explained in depth in the the [FFL report](https://ff06-2020.fastforwardlabs.com/).
For more information, refer to the [lime documentation](https://lime-ml.readthedocs.io/en/latest/lime.html).

In [ ]:
from lime.lime_tabular import LimeTabularExplainer

data[labels.name + ' probability'] = pipe.predict_proba(X)[:, 1]

# List of length number of features, containing names of features in order
# in which they appear in X
feature_names = list(ce.columns_)

# List of indices of columns of X containing categorical features
categorical_features = list(ce.cat_columns_ix_.values())

# List of (index, [cat1, cat2...]) index-strings tuples, where each index
# is that of a categorical column in X, and the list of strings are the
# possible values it can take
categorical_names = {i: ce.classes_[c]
                     for c, i in ce.cat_columns_ix_.items()}
class_names = ['No ' + labels.name, labels.name]
explainer = LimeTabularExplainer(ce.transform(data),
                                 feature_names=feature_names,
                                 class_names=class_names,
                                 categorical_features=categorical_features,
                                 categorical_names=categorical_names)    



## Explaining a Single Prediction

Let's look at how one specfic prediction would be interpreted.
Lime explains the prediction by giving every feature a weight from -1 to 1.
Features with weights closer to -1 have a stronger impact in coming up with a 0 prediction result (will not churn) and vice versa.

In [ ]:
data.sample().T  # reminder of the features

In [ ]:
exp = explainer.explain_instance(ce.transform(data.sample())[0],pipe.predict_proba)
for cols in exp.as_list():
    print(cols)

In [ ]:
exp.as_pyplot_figure()
plt.tight_layout()

We see that one of the features that contributes most strongly to the positive prediction is the short tenure of the customer.

## Saving the model
Now that we've done all this work to build the models, we want to be able to use them later.
The `ExplainedModel` class is a handy wrapper for using the `CategoricalEncoder`, the `Pipeline` object which *is* the churn model, and the Lime Explainer.
Here, we use it to save these trained models for use in later parts of the Project.

In [ ]:
from churnexplainer import ExplainedModel
explainedmodel = ExplainedModel(data=data, labels=labels, model_name='telco_linear',
                                categoricalencoder=ce, pipeline=pipe,
                                explainer=explainer,data_dir=data_dir)
explainedmodel.save()

In [ ]:
spark.stop()

## Wrap up
We've now covered all the steps to **building a machine learning model** including interpretability
and saved our work for use in later sections.

In the next part of the series we will explore how to use the **Experiments** feature of CML
for when we want to test lots of combinations of hyperparameters to fine tune our models.
